## How can we calculate the entropy of a solid?



A common task in thermodynamics is calculating the entropy of a substance. Because nobody has invented a device that can measure entropy experimentally, we resort to other measurable quantities (heat capacity and enthalpy) and mathematical relationships. Knowing entropy, it is possible to identify what processes will occur spontaneously and predict when they will reach equilibrium. In this activity, you will calculate the entropy of a substance in a single phase.



### Prerequisites



This notebook assumes users are familiar with thermodynamics concepts including

-   differential forms for the first and second laws of thermodynamics
-   temperature dependence of heat capacity

and python/Jupyter skills

-   importing and parsing CSV files
-   basic plotting with matplotlib
-   creating and manipulating numpy arrays
-   using Scipy numerical integration functions



### Content Objectives



After completing this exercise you should be able to

-   apply the first and second laws of thermodynamics for calculating the entropy of a substance in a single phase
-   relate heat capacity to entropy mathematically
-   predict the effect of structure on the entropy of a substance



### Process Objectives



After completing this exercise you should be able to use python to

-   import and visualize structured data
-   fit experimental values to a polynomial
-   assess the quality of fit using statistical measures RSS or SSE



### Python Environment



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial.polynomial import polyfit, polyval, Polynomial
from scipy.integrate import trapezoid, simpson, quad

WU = Warm Up questions; CTQ = Critical Thinking Questions.



## Warm Up Questions



Answer questions individually, then compare and discuss answers with your group until there is consensus.

**WU 1** List the devices and methods that can be used to measure a) enthalpy changes b) heat capacity, and c) entropy changes.



**WU 2** How is heat capacity measured experimentally?



**WU 3** Using words, write a working definition of the third law of thermodynamics.



**WU 4** Using words, write a working definition of entropy. Is entropy temperature dependent?



**WU 5** Write a differential expression for the second law of thermodynamics, relating $S$, $q$, and $T$.



**WU 6** Write the relationship that connects $S(T)$ and $C_p$ over the temperature range $T_i$ to $T_f$.



**WU 7** Using the relationship in the previous question for inspiration, propose an experiment that would allow you to measure the entropy of a substance in the lab.



**WU 8**  Predict whether the isobaric heat capacity, $C_p$, for a substance in a given phase should increase, decrease, or remain constant as $T$ increases. Explain your reasoning.



## How to calculate the heat capacity of Ag(s) as a function of temperature



Procedure for modeling $C_p(T)$, which can then be used to calculate molar entropy:

1.  import and fit experimental $C_p$ data to a polynomial
2.  assess quality of fit using [Residual Sum of Squares](https://en.wikipedia.org/wiki/Residual_sum_of_squares) (RSS), also known as the Sum of Squares Error (SSE)
3.  calculate $S(T)$ over experimental temperature range, $S(T) = \int_{T_i}^{T_f} \frac{C_P}{T}\; dT$
4.  apply Debye approximation for heat capacity at low temperatures



### Import and visualize experimental data



Examine the data file for headers and format.



In [ ]:
!head -n 5 cpSilver.csv

**CTQ 1** What are the data, units, and delimiters in this file?



**Exercise 1**
Modify this code block to import the silver heat capacity data into numpy arrays and plot the data to visualize the temperature dependence.



In [ ]:
# Ts, Cps = np.genfromtxt('filename.csv', delimiter='some delimiter', unpack=True, skip_header=1)
# ...

**CTQ 2** a) Does the plot agree with your prediction in WU-8? b) Does your group think the curve has a recognizable shape (e.g., quadratic, exponential, reciprocal)?



**CTQ 3** In your paper notebook or Jupyter notebook, capture the steps used and any &ldquo;gotchas&rdquo; for visualizing experimental data using python.



### Fitting $C_p$ data to a polynomial



**Worked Example**   
Plan: Fit $C_p$ data to a polynomial of degree $n$ and plot the fit with experimental data. Assess the quality of fit using [RSS](https://en.wikipedia.org/wiki/Residual_sum_of_squares) and the variance. Smaller values for RSS and variance suggest a better fit to the data.



In [ ]:
n = 2  # Cp vs T doesn't look linear, so n=1 is not a good choice.
fit = polyfit(Ts, Cps, n)
fitvalues = polyval(Ts, fit)

rss = np.sum((Cps - fitvalues)**2)
variance = rss / (len(Cps)-1)
print(f"n={n}, rss={rss:.2f}, variance={variance:.2e}")

fig, ax = plt.subplots()
ax.plot(Ts, Cps, 'o')
ax.plot(Ts, fitvalues)

The optimized polynomial coefficients are stored in the array named &ldquo;fit&rdquo;. The coefficients can be printed:



In [ ]:
print(fit)
print(Polynomial(fit))

**CTQ 4** Examine the polynomial equation that polyfit found with $n=2$. Does the equation reflect the general trend of the data? How would you assess the quality of the fit?



**CTQ 5** From the code above, write the mathematical expressions for RSS and variance. Identify each variable (explain what each variable is).



**CTQ 6** How can RSS and variance be used to assess the quality of the fit?



**CTQ 7** Based on the information above, could the heat capacity data be fit to a 12th order polynomial (n=12)?



**Exercise 2**  
Vary the value of $n$ and find a polynomial that fits the data well. Use the minimum number of fit parameters (terms in the polynomial) to obtain reasonable agreement.

Tabulate n and goodness-of-fit measures for at least 5 different polynomials.



**CTQ 8**
 Give the full mathematical expression for the polynomial that you chose as having the best fit, and explain your reasoning for choosing it.



**CTQ 9** Are there disadvantages to fitting the heat capacity data to higher order polynomials (for example, 12th order)?



### How many fitting parameters should be used?



In experimental settings, [overfitting](https://en.wikipedia.org/wiki/Overfitting)  refers to the situation where a statistical model or analysis captures noise or random fluctuations in the data rather than the true underlying relationships or patterns. Two common ways overfitting can manifest in experimental data:

-   Model Complexity: Using overly complex models (e.g., high-degree polynomials or intricate statistical models) can lead to fitting the noise in the data. This can result in misleading conclusions about the relationships between variables.
-   Small Sample Sizes: When the sample size is small, there is a higher risk of overfitting because the model may be too closely tailored to the specific data points available.

**Exercise 3**  
Plot the model function polynomial with n=7 from 10K to 300K and from 10K to 400K or more. (Offset one of the curves by adding an arbitrary constant if you want to plot both curves on the same axes.)



**CTQ 10**  Assess the ability of the n=7 polynomial model to predict $Cp$ at temperatures above 300K. What might be limiting the predictive ability of the model?



**CTQ 11** Are there disadvantages to fitting the heat capacity data to a high-order polynomial?



### Calculate entropy over the measured temperature range



**CTQ 12** Write the expression relating $S$ to $C_p(T)$.



**Exercise 4**  
Use the best-fit polynomial you chose above to calculate $S$ over the measured temperature range. The functions [trapezoid](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.trapezoid.html#scipy.integrate.trapezoid), [simpson](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.simpson.html#scipy.integrate.simpson), and [quad](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html#scipy.integrate.quad) all perform numerical integration. Giauque reports a value around 40 J/mol K using graphical integration.



### Adding the Debye extrapolation



Note there are no experimental data below about 10K. Because it is difficult to measure heat capacity at low temperatures (how would you design a calorimeter that works at a few Kelvin?), the [Debye model](https://en.wikipedia.org/wiki/Debye_model) is often used to cover temperatures from 0 K to the lowest temperature that can be measured experimentally.

At low temperatures, many substances follow the relationship
$$C_p = aT^3 $$
where $a$ is an empirical constant.

**CTQ 13** Show that
$$ S(T_{low}) = \dfrac{aT_{low}^3}{3}$$

**Exercise 5**  
Using $C_p$ at $T_{low}$, determine $a$ for silver from the data. Next, calculate $S$ from 0 K to the lowest measured temperature in the data.



### The whole picture



**Exercise 6**  
Calculate the total entropy from 0K to the maximum temperature in the data.



**CTQ 14** Find the molar entropy for Ag at 298.15K on the NIST Webbook or in the CRC Handbook and compare it to your result. Quantify any differences and assess the quality of the modeling approach that was used.



**CTQ 15** Could this modeling approach be used to calculate the entropy of Ag at, say, 500K? At 1500K? Why or why not?



## How do the entropies of other substances compare to Ag?



Each person in the group choose one element from beryllium, gold, or copper (double up if necessary). 

**CTQ 16** Make a prediction: How will the molar entropy of your element compare to that of Ag over the same temperature range? Explain your reasoning.



**CTQ 17** As a group (without referring to outside sources) arrange the elements in order of *increasing* molar entropy. Explain your reasoning.



## What is the entropy of my element?



**Exercise 7**  
Use Cp/temperature data to determine the molar entropy of your element from 0 to 298K. This cell will list the files accessible by Jupyter. You may need to upload the associated data file if it does not appear in the list.



In [ ]:
# list csv data files
import glob; glob.glob("*.csv")

In [ ]:
# enter your code for the entropy calculation

**CTQ 18** Compare the entropy to that found on the NIST Webbook or in the CRC Handbook and to your prediction. Cite your reference.



**CTQ 19** Pool the groups results for all of the elements. Was the predicted order correct? if not, what may have led to an unsupported prediction.



**CTQ 20** Briefly summarize the mathematical relationships and process used to obtain the molar entropy of a substance in a single phase.



## Reflection



On your own, review the Content and Process Objectives at the beginning of the activity. Identify which of the objectives you feel you meet and those that may need more work; explain reasoning for your assessment.



Identify parts of the activity that stood out as surprising, interesting, or confusing, and explain why.



## References



Engle and Reid, *Physical Chemistry*, 3rd ed., Pearson, 2013.

McQuarrie and Simon, *Physical Chemistry: A Molecular Approach*, University Science Books, 1997

T. Grubbs, LibreTexts: Data Driven Exercises [Molar Enthalpy of Al2O3](https://chem.libretexts.org/Bookshelves/Physical_and_Theoretical_Chemistry_Textbook_Maps/Exercises%3A_Physical_and_Theoretical_Chemistry/Data-Driven_Exercises/Standard_Molar_Entropy_of_Aluminum_Oxide) (accessed 2025-06)

Hamilton and Zielinski, &ldquo;Conventional Entropies and the Third Law of Thermodynamics&rdquo;, Symmath Project (defunct)



## Acknowledgement



[ACT-CMS](https://act-cms.molssi.org/) is funded by the National Science Foundation Training-based Workforce Development for Advanced Cyberinfrastructure program (OAC 2321044,OAC 2321045).

